# VIC Conservation Status and Sensitive Species Lists

This notebook downloads the VIC lists from the [Qld Government Open Data Portal](https://vba.dse.vic.gov.au/vba/downloadVSC.do) and formats them in Darwin Core for ingestion into the ALA Lists tool.
It will save original lists to the `source-data/ViC` directory, process the lists and save them to `current-lists`.


## Lists in the ALA Species List tool
* Conservation list: __[dr655](https://lists.ala.org.au/speciesListItem/list/dr655)__ (in [test](https://lists-test.ala.org.au/speciesListItem/list/dr655))
* Sensitive list: __[dr490](https://lists.ala.org.au/speciesListItem/list/dr490)__ (in [test](https://lists-test.ala.org.au/speciesListItem/list/dr490))


## Sources
Victorian Biodivesity Atlas - Victoria Department of Environment, Land, Water and Planning

### Conservation
* __[Metadata - xxx (Open Data Portal)](https://)__
* __[Data](https://vba.dse.vic.gov.au/vba/downloadVSC.do)__

### Sensitive
* __[Metadata - xxx (Open Data Portal)](https://)__
* __[Data](https://vba.dse.vic.gov.au/vba/downloadVSC.do)__

### Codes
* __[Metadata - xxxxx](https://)__
* __[Data](https://)__


In [1]:
# import function definitions from includes directory
%cd ..
from includes import list_functions

C:\Users\oco115\PycharmProjects\authoritative-lists\source-code


In [2]:
import pandas as pd

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
cdataDir = "current-lists/conservation-lists/"
sdataDir = "current-lists/sensitive-lists/"
conservationlisturl = "https://vba.dse.vic.gov.au/vba/downloadVSC.do" # gets a csv
conservationList = pd.read_csv(projectDir + "source-data/VIC/Species-Checklist.csv",encoding='cp1252')


## Extract conservation list
1. Drop non-required columns
2. Rename columns for Darwin Core list terms
3. Assign additional required list fields

In [3]:
# Drop columns
conservationList = conservationList.drop(['PARENT_TAXON_LEVEL_CDE','PARENT_TAXON_ID','ALL_DISCIPLINE_CODES',
                                          'TREATIES','SHORT_NAME','NVIS_GROWTHFORM','SCIENTIFIC_NME_SYNONYM',
                                          'COMMON_NME_SYNONYM','PRINT_ORDER_NUM'],axis=1)
# Rename columns
conservationList = conservationList.rename(columns=
{
    'TAXON_ID':'taxonID',
    'SCIENTIFIC_NAME':'scientificName',
    'VIC_ADVISORY_STATUS':'vicadvisorystatus',
    'TAXON_TYPE':'speciesGroup',
    'FFG_ACT_STATUS':'ffgactstatus',
    'EPBC_ACT_STATUS': 'epbcactStatus',
    'TAXON_LEVEL_CDE': 'taxonRank',
    'ORIGIN': 'establishmentMeans',
    'COMMON_NME_SYNONYM': 'taxonRemarks',
    'COMMON_NAME':'vernacularName',
    'AUTHORITY':'scientificNameAuthority',
    'PRIMARY_DISCIPLINE': 'primaryDiscipline',
    'RESTRICTED_FLAG' : 'restrictedFlag',
    'EXTRACT_DATE': 'extractDate',
    'LAST_MOD':'modified'
})
conservationList['taxonRank'] = 'species'
conservationList['status'] = conservationList['ffgactstatus']
conservationList['sourceStatus'] = conservationList['ffgactstatus']

In [4]:
# reformat date
conservationList['extractDate'] = pd.to_datetime(conservationList['extractDate'], format='%d/%m/%Y')
conservationList['extractDate'] = conservationList['extractDate'].dt.strftime('%Y-%m-%d')
conservationList['modified'] = pd.to_datetime(conservationList['modified'], format='%d/%m/%Y')
conservationList['modified'] = conservationList['modified'].dt.strftime('%Y-%m-%d')
sensitiveList = conservationList
conservationList = conservationList[conservationList["status"].notna()]

conservationList.head(4)

,taxonID,scientificName,vernacularName,scientificNameAuthority,primaryDiscipline,establishmentMeans,taxonRank,speciesGroup,ffgactstatus,epbcactStatus,vicadvisorystatus,restrictedFlag,modified,extractDate,status,sourceStatus
9,4864,Ambassis agassizii,Agassiz's Glassfish,"Steindachner, 1867",Aquatic fauna,NaN,species,Fish,Extinct,NaN,Regionally extinct,NaN,2013-04-04,2023-01-16,Extinct,Extinct
28,528544,Bidyanus bidyanus,Silver Perch,"(Mitchell, 1838)",Aquatic fauna,NaN,species,Fish,Endangered,Critically Endangered,Vulnerable,NaN,2016-05-24,2023-01-16,Endangered,Endangered
37,5133,Chelodina expansa,Broad-shelled Turtle,"Gray, 1857",Aquatic fauna,NaN,species,Reptiles,Endangered,NaN,Endangered,NaN,2014-11-20,2023-01-16,Endangered,Endangered
48,4784,Craterocephalus fluviatilis,Murray Hardyhead,"McCulloch, 1912",Aquatic fauna,NaN,species,Fish,Critically Endangered,Endangered,Critically endangered,NaN,2013-04-04,2023-01-16,Critically Endangered,Critically Endangered


## Extract Sensitive List
1. Using the conservationList extract Sensitive list records
2. Records where RESTRICTED_FLAG is not null

In [5]:
sensitiveList = sensitiveList[sensitiveList["restrictedFlag"].notna()]
len(sensitiveList.index)

136

In [6]:
#Write to CSV
conservationList.to_csv(projectDir + cdataDir + "VIC-conservation.csv",index=False)
sensitiveList.to_csv(projectDir + sdataDir + "VIC-sensitive.csv",index=False)
print('Finished processing')

Finished processing


# Manual List check

**Instructions**
1. Load the lists above into the lists-test tool
2. Check the list name matching score and the text appearance on species pages
3. Unskip the below code and Run the reports below to compare to production. Send the changelog.csv to check. Correct any issues.
4. Save the production list into the `historical lists` directory by uncommenting the code section below.
5. Load the lists into production

### Conservation List - Download old and new and compare

In [13]:
# %%script echo skipping # comment this line to run this code

import datetime
monthStr = datetime.datetime.now().strftime('%Y%m')

# conservation
filename = "VIC-conservation.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr655" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr655" + "?max=10000&includeKVP=true"
changelist = list_functions.get_changelist(testListUrl, prodListUrl)
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
prodList = list_functions.download_ala_list(prodListUrl) # save the prod list to the historical lists directory
prodList = list_functions.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/conservation/" + filename, encoding="UTF-8", index=False)
changelist

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
36,Galaxias sp. 14,Galaxias,Galaxias terenasus,Critically Endangered,added,NaN,NaN,NaN
77,Hyridella (Hyridella) depressa,Freshwater Mussel,Hyridella (Hyridella) depressa,Endangered,added,NaN,NaN,NaN
79,Hyridella narracanensis,Freshwater Mussel,Hyridella (Hyridella) narracanensis,Endangered,added,NaN,NaN,NaN
985,Leichhardtia australis,Doubah,Marsdenia australis,Endangered,added,NaN,NaN,NaN
986,Leichhardtia flavescens,None,Leichhardtia flavescens,Critically Endangered,added,NaN,NaN,NaN
1148,Orianthera pusilla,None,Orianthera pusilla,Vulnerable,added,NaN,NaN,NaN
1317,Prasophyllum sp. aff. montanum,None,Prasophyllum sp. montane (D.T.Rouse 180),Endangered,added,NaN,NaN,NaN
1320,Prasophyllum sp. aff. validum,None,Prasophyllum sp. Woodlands (D.L.Jones 17707),Endangered,added,NaN,NaN,NaN
1039,Logania pusilla,NaN,NaN,NaN,removed,None,Orianthera pusilla,Vulnerable
1060,Marsdenia australis,NaN,NaN,NaN,removed,Doubah,Marsdenia australis,Endangered


### Sensitive List - Download old and new and compare

In [14]:
# %%script echo skipping # comment this line to run this code
filename = "VIC-sensitive.csv"
prodListUrl = "https://lists.ala.org.au/ws/speciesListItems/" + "dr490" + "?max=10000&includeKVP=true"
testListUrl = "https://lists-test.ala.org.au/ws/speciesListItems/" + "dr18669" + "?max=10000&includeKVP=true"
changelist = list_functions.get_changelist(testListUrl, prodListUrl )
# save the lists locally
changelist.to_csv(projectDir + "analysis/change-log/" + monthStr + "-" + filename, encoding="UTF-8", index=False)
prodList = list_functions.download_ala_list(prodListUrl) # save the prod list to the historical lists directory
prodList = list_functions.kvp_to_columns(prodList)
prodList.to_csv(projectDir + "historical-lists/sensitive/" + filename, encoding="UTF-8", index=False)
changelist
## check this as no list in test at the moment

,name,commonName_new,scientificName_new,status_new,listUpdate,commonName_old,scientificName_old,status_old
104,Miniopterus orianae bassanii,Southern Bent-winged Bat,Miniopterus orianae bassanii,Critically Endangered,added,NaN,NaN,NaN
38,Wundacaenis flabellum,NaN,NaN,NaN,removed,Mayfly,Wundacaenis flabellum,Vulnerable
39,Westriplectes angelae,NaN,NaN,NaN,removed,Caddisfly,Westriplectes angelae,Vulnerable
40,Oecetis quadrula,NaN,NaN,NaN,removed,Caddisfly,Oecetis quadrula,Vulnerable
41,Pseudocloeon hypodelum,NaN,NaN,NaN,removed,Mayfly,Pseudocloeon hypodelum,Vulnerable
44,Pterostylis tenuissima,NaN,NaN,NaN,removed,Swamp Greenhood,Pterostylis tenuissima,Vulnerable
45,Caladenia tensa,NaN,NaN,NaN,removed,Erect Greencomb Spider Orchid,Caladenia tensa,Vulnerable
60,Prasophyllum sp. aff. validum A,NaN,NaN,NaN,removed,None,Prasophyllum sp. Woodlands (D.L.Jones 17707),Endangered
101,Thaumatoperla robusta,NaN,NaN,NaN,removed,Stonefly,Thaumatoperla robusta,Data Deficient
102,Thaumatoperla timmsi,NaN,NaN,NaN,removed,Stonefly,Thaumatoperla timmsi,Data Deficient
